# 1. Create V, C

In [1]:
import nltk
import numpy as np
from nltk.corpus import brown, stopwords
from nltk.tokenize import word_tokenize
from math import log

In [2]:
nltk.download('brown')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\pin_t\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pin_t\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pin_t\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
words = ' '.join(brown.words())
print (words[:200])

The Fulton County Grand Jury said Friday an investigation of Atlanta's recent primary election produced `` no evidence '' that any irregularities took place . The jury further said in term-end present


In [4]:
from collections import defaultdict, Counter
stop = set(stopwords.words('english'))
wordsNew = [word for word in word_tokenize(words.lower()) if word not in stop and word.isalpha()]
wordsNew = np.asarray(wordsNew)

In [5]:
len(wordsNew)

515753

In [6]:
wordsCount = Counter(wordsNew)
V = np.asarray([e[0] for e in wordsCount.most_common(5000)])
C = V[:2000]

# 2. Calculate P(c) and P(c|w)

In [7]:
def word2map(words):
    word2idx = defaultdict(int);
    for word in words:
        word2idx[word] = len(word2idx)
    return word2idx

In [8]:
V2idx = word2map(V);
C2idx = word2map(C);

In [9]:
V_SIZE = len(V)
C_SIZE = len(C)
HALF_WINDOW = 2
M = len(wordsNew)

# prepare for the calculation of Pr(c) and Pr(c|w)
# use ones to apply laplace smoothing
print ("counting context appearance...")
window_count = np.ones((V_SIZE, C_SIZE))
core_count = np.ones((1, C_SIZE))
for i in range(M):
    w = wordsNew[i]
    if not w in V2idx:
        continue
    else:
        wid = V2idx[w]
    for j in range(i - HALF_WINDOW, i + HALF_WINDOW + 1):
        if j < 0 or j >= M or j == i:
            continue
        c = wordsNew[j]
        if not c in C2idx:
            continue
        else:
            cid = C2idx[c]
        window_count[wid][cid] += 1
        core_count[0][cid] += 1
print('done')

counting context appearance...
done


In [10]:
# calculate Pr(c) and Pr(c|w)
print ("calculating probability...")
pwc, pc = window_count, core_count
for i in range(len(pwc)):
    pwc[i] = pwc[i] / pwc[i].sum()
pc = pc / pc.sum()

# calculate pointwise mutual information
r = np.zeros((V_SIZE, C_SIZE))
for i in range(V_SIZE):
    for j in range(C_SIZE):
        r[i][j] = max(0, log(pwc[i][j] /  pc[0][j]))

# save representation matrix to file
print ("saving representation...")
np.save("representation-" + str(C_SIZE) + ".npy", r)
print('done')

calculating probability...
saving representation...
done


# 3. Dimension Reduction

In [12]:
from sklearn.decomposition import PCA
U_SIZE = 100
r = np.load("representation-" + str(C_SIZE) + ".npy");
pca = PCA(n_components = U_SIZE);
cr = pca.fit_transform(r);
np.save("representation-" + str(U_SIZE) + ".npy", cr);

# 4. Clustering

In [13]:
import random
from numpy.linalg import norm

from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity

from queue import PriorityQueue

In [14]:
def map2word(words):
    idx2word = defaultdict(str);
    for i, word in enumerate(words):
        idx2word[i] = word
    return idx2word

In [15]:
idx2V = map2word(V)
idx2C = map2word(C)

In [16]:
CLUSTER_NUMBER = 100
CLUSTER_THRESHOLD = 20
# cluster words
X = np.load("representation-" + str(U_SIZE) + ".npy")
kmeans = KMeans(n_clusters = CLUSTER_NUMBER).fit(X)

# sort words by distance to center
word_groups = {i:PriorityQueue() for i in range(CLUSTER_NUMBER)}
for i in range(len(X)):
    representation = X[i]
    word = idx2V[i]
    center_id = kmeans.predict(X[i].reshape(1, -1))[0]
    dist = norm(representation - kmeans.cluster_centers_[center_id])
    word_groups[center_id].put((float(dist), word))

# print only relatively large groups
for i in range(CLUSTER_NUMBER):
    if word_groups[i].qsize() < CLUSTER_THRESHOLD:
        continue
    count = 0
    for j in range(CLUSTER_THRESHOLD):
        if word_groups[i].empty():
            break
        print (word_groups[i].get()[1],)
        count += 1
        if count % 10 == 0:
            print
    print ("\n******************************")

jim
bobbie
damn
recall
intelligent
drunk
hated
dawn
succeeded
mistake
jersey
tim
hoping
nowhere
visiting
chicken
worst
violent
doctors
libraries

******************************
question
change
rather
matter
thus
problem
means
example
policy
problems
experience
field
human
action
community
whole
important
economic
interest
point

******************************
activity
distribution
mass
indicated
applied
latter
maximum
analysis
obtained
parts
materials
unit
rise
figures
described
cells
somewhat
gas
showed
using

******************************
revenues
surplus
districts
safety
transportation
concerns
abroad
amounts
finance
farmers
shooting
estate
contribute
hopes
devoted
developments
legislation
financing
difficulties
electronics

******************************
difficult
believe
hope
therefore
century
idea
view
freedom
indeed
attention
england
seem
common
able
spirit
true
nations
future
society
america

******************************
bag
shorts
burned
instant
stranger
ramey
noticed
watso

# 5. Nearest Neighbor

In [17]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import normalize


# use cosine distance
tr = 1 - cosine_similarity(np.load("representation-" + str(U_SIZE) + ".npy"))
np.fill_diagonal(tr, 0)

neigh = KNeighborsClassifier(n_neighbors = 2, metric = "precomputed")
neigh.fit(tr, np.zeros((V_SIZE)))

# find NN for 25 random words
rand_inds = random.sample([i for i in range(V_SIZE)], 25)
for i in rand_inds:
    w = idx2V[i]
    dist, ind = neigh.kneighbors(tr[i].reshape(1, -1))
    uid = ind[0][1]
    u = idx2V[uid]
    print ("%-15s %-15s %f" %(w, u, dist[0][1]))

era             chorus          0.335989
submarine       dynamic         0.253552
changes         change          0.486755
birth           death           0.517800
sending         al              0.308579
divorce         dick            0.266110
cousin          heaven          0.460679
assembly        mayor           0.516453
language        literature      0.425120
silent          stomach         0.382788
evil            bad             0.441725
educated        wit             0.186698
minimal         polynomial      0.185531
founded         crown           0.127797
florida         di              0.129886
wagon           parked          0.383355
refer           h               0.287761
arrive          franklin        0.246195
explain         tried           0.480053
examination     theories        0.314438
cure            screw           0.191759
span            preserved       0.169148
automatically   pointing        0.285135
painter         meredith        0.201946
looks           